# Анализ результатов экспериментов по классификации страховых писем

Данный notebook предназначен для интерактивного анализа результатов экспериментов.

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Добавляем путь к проекту
project_root = Path('../')
sys.path.insert(0, str(project_root))

from src.data.data_loader import InsuranceDataLoader
from src.data.data_analyzer import InsuranceLetterAnalyzer
from src.utils.visualization import plot_class_distribution, generate_results_table

plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

## 1. Загрузка и анализ данных

In [ ]:
# Загрузка данных
data_loader = InsuranceDataLoader()
data_path = '../data/raw'

train_df, test_df, services_df = data_loader.load_data(data_path)
train_df = data_loader.preprocess_letters(train_df)
test_df = data_loader.preprocess_letters(test_df)

print(f"Обучающая выборка: {len(train_df)} примеров")
print(f"Тестовая выборка: {len(test_df)} примеров")
print(f"Справочник услуг: {len(services_df)} услуг")

In [ ]:
# Анализ распределения классов
class_counts = train_df['class'].value_counts()
print("Распределение классов:")
print(f"Класс 0 (общие запросы): {class_counts[0]} ({class_counts[0]/len(train_df)*100:.1f}%)")
print(f"Класс 1 (с кодами услуг): {class_counts[1]} ({class_counts[1]/len(train_df)*100:.1f}%)")
print(f"Дисбаланс классов: {class_counts[0]/class_counts[1]:.1f}:1")

# Визуализация
plot_class_distribution(train_df, title="Распределение классов в обучающей выборке")
plt.show()

## 2. Анализ текстовых характеристик

In [ ]:
# Анализ длины текстов по классам
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Длина символов
train_df.boxplot(column='text_length', by='class', ax=ax1)
ax1.set_title('Длина текста (символы) по классам')
ax1.set_xlabel('Класс')
ax1.set_ylabel('Количество символов')

# Количество слов
train_df.boxplot(column='num_words', by='class', ax=ax2)
ax2.set_title('Количество слов по классам')
ax2.set_xlabel('Класс')
ax2.set_ylabel('Количество слов')

plt.tight_layout()
plt.show()

In [ ]:
# Статистика по кодам услуг
print("Статистика по кодам медицинских услуг:")
print(f"Всего писем с кодами: {train_df['has_service_codes'].sum()}")
print(f"Среднее количество кодов на письмо: {train_df[train_df['num_service_codes'] > 0]['num_service_codes'].mean():.2f}")
print(f"Максимальное количество кодов: {train_df['num_service_codes'].max()}")

# Распределение количества кодов
codes_dist = train_df['num_service_codes'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
codes_dist.plot(kind='bar')
plt.title('Распределение количества кодов услуг в письмах')
plt.xlabel('Количество кодов')
plt.ylabel('Количество писем')
plt.show()

## 3. Анализ результатов экспериментов

**Примечание**: Для анализа результатов сначала запустите эксперименты с помощью `python main.py`

In [ ]:
# Загрузка результатов последнего эксперимента
import json
import glob

# Найти последний эксперимент
results_dir = Path('../results')
experiment_dirs = list(results_dir.glob('experiment_*'))

if experiment_dirs:
    latest_experiment = max(experiment_dirs, key=lambda p: p.stat().st_mtime)
    results_file = latest_experiment / 'experiment_results.json'
    
    if results_file.exists():
        with open(results_file, 'r', encoding='utf-8') as f:
            results = json.load(f)
        
        print(f"Загружены результаты из: {latest_experiment}")
        print(f"Дата эксперимента: {results['experiment_info']['timestamp']}")
    else:
        print("Файл результатов не найден. Запустите эксперименты: python main.py")
        results = None
else:
    print("Результаты экспериментов не найдены. Запустите: python main.py")
    results = None

In [ ]:
# Анализ результатов, если они доступны
if results and 'experiments' in results:
    experiments = results['experiments']
    
    # Создание сводной таблицы
    summary_data = []
    
    if 'baseline' in experiments:
        baseline = experiments['baseline']
        summary_data.append({
            'Эксперимент': 'Базовая модель',
            'F1-score': f"{baseline['test_metrics']['f1_weighted']:.3f}",
            'Accuracy': f"{baseline['test_metrics']['accuracy']:.3f}",
            'Precision': f"{baseline['test_metrics']['precision_weighted']:.3f}",
            'Recall': f"{baseline['test_metrics']['recall_weighted']:.3f}"
        })
    
    if 'synthetic_augmentation' in experiments:
        for exp_name, exp_data in experiments['synthetic_augmentation'].items():
            summary_data.append({
                'Эксперимент': f'Синтетические ({exp_data["synthetic_size"]})',
                'F1-score': f"{exp_data['test_metrics']['f1_weighted']:.3f}",
                'Accuracy': f"{exp_data['test_metrics']['accuracy']:.3f}",
                'Precision': f"{exp_data['test_metrics']['precision_weighted']:.3f}",
                'Recall': f"{exp_data['test_metrics']['recall_weighted']:.3f}"
            })
    
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)
else:
    print("Данные экспериментов недоступны для анализа.")

In [ ]:
# Визуализация улучшений от синтетических данных
if results and 'experiments' in results and 'synthetic_augmentation' in results['experiments']:
    synthetic_results = results['experiments']['synthetic_augmentation']
    baseline_f1 = results['experiments']['baseline']['test_metrics']['f1_weighted']
    
    sizes = []
    f1_scores = []
    improvements = []
    
    for exp_data in synthetic_results.values():
        size = exp_data['synthetic_size']
        f1 = exp_data['test_metrics']['f1_weighted']
        improvement = ((f1 - baseline_f1) / baseline_f1) * 100
        
        sizes.append(size)
        f1_scores.append(f1)
        improvements.append(improvement)
    
    # Сортировка по размеру
    sorted_data = sorted(zip(sizes, f1_scores, improvements))
    sizes, f1_scores, improvements = zip(*sorted_data)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # График F1-score
    ax1.plot(sizes, f1_scores, 'o-', linewidth=2, markersize=8, label='Синтетические данные')
    ax1.axhline(y=baseline_f1, color='r', linestyle='--', label='Базовая модель')
    ax1.set_title('Влияние размера синтетических данных на F1-score')
    ax1.set_xlabel('Размер синтетических данных')
    ax1.set_ylabel('F1-score')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # График улучшений
    ax2.plot(sizes, improvements, 'o-', linewidth=2, markersize=8, color='green')
    ax2.set_title('Относительное улучшение F1-score')
    ax2.set_xlabel('Размер синтетических данных')
    ax2.set_ylabel('Улучшение (%)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Выводы
    best_idx = np.argmax(f1_scores)
    print(f"\nЛучший результат достигнут с {sizes[best_idx]} синтетическими примерами:")
    print(f"F1-score: {f1_scores[best_idx]:.3f} (улучшение на {improvements[best_idx]:.1f}%)")

## 4. Выводы

Основываясь на проведенном анализе, можно сделать следующие выводы:

1. **Дисбаланс классов**: В исходных данных наблюдается значительный дисбаланс классов (~2.7:1)
2. **Синтетические данные**: Генерация синтетических данных помогает улучшить качество классификации
3. **Оптимальный размер**: Оптимальный размер синтетической выборки определяется экспериментально

**Для получения полных результатов запустите эксперименты:**
```bash
python main.py --mode full
```